In [6]:
from glob import glob
import cv2

def get_videos(vid_id, kernel = None):
    if kernel == None:
        videos = [f'/home/luke/Desktop/project/make_tea/camera-main/videos/{vid_id}/left/{vid_id}-left.mp4',
          f'/home/luke/Desktop/project/make_tea/camera-main/videos/{vid_id}/right/{vid_id}-right.mp4']
    else:
        if not isinstance(kernel, list):
            suffix = f'_{kernel}'
        else:
            temp = '_'.join(kernel)
            suffix = f'_{temp}'
        videos = [f'/home/luke/Desktop/project/make_tea/camera-main/videos/{vid_id}/left/{vid_id}-left{suffix}.mp4',
          f'/home/luke/Desktop/project/make_tea/camera-main/videos/{vid_id}/right/{vid_id}-right{suffix}.mp4']
    return videos

def get_h5files(vid_id, kernel): 
    if kernel == None:
        h5files = glob(f'/home/luke/Desktop/project/make_tea/camera-main/videos/{vid_id}/left/{vid_id}-leftDLC*.h5')+\
        glob(f'/home/luke/Desktop/project/make_tea/camera-main/videos/{vid_id}/left/{vid_id}-leftDLC*.h5')
    else:
        if not isinstance(kernel, list):
                suffix = f'_{kernel}'
        else:
            temp = '_'.join(kernel)
            suffix = f'_{temp}'
        h5files = [f'/home/luke/Desktop/project/make_tea/camera-main/videos/{vid_id}/left/{vid_id}-left{suffix}.h5',
          f'/home/luke/Desktop/project/make_tea/camera-main/videos/{vid_id}/right/{vid_id}-right{suffix}.h5']
    return h5files

def video_to_frames(video_path):
    
    vidcap = cv2.VideoCapture(video_path)
    success,image = vidcap.read()
    frames = []
    while success:
      frames.append(image)      
      success,image = vidcap.read()
    print(f'The number of frames is {len(frames)}')
    return frames
    

In [88]:
vid_id = '1642994619'
kernel = ['nearest', 'median']
# kernel = None
videos = get_videos(vid_id, kernel)
h5files = get_h5files(vid_id, kernel)
frames_left = video_to_frames(videos[0])
frames_right = video_to_frames(videos[1])
suffix = '_'.join(kernel)



The number of frames is 538
The number of frames is 538


NameError: name 'kernels' is not defined

In [89]:
suffix = '_'.join(kernel)
df = pd.read_hdf(h5files[0])
df_new = df.copy()
df_new.columns = df_new.columns.set_levels(['z', 'x', 'y'], level='coords')

In [90]:
x_inds = df_new.columns.values[::3]
y_inds = df_new.columns.values[1::3]
z_inds = df_new.columns.values[2::3]
for i in range(len(frames_left)):
    data = df.iloc[i]
    x = data[x_inds].values
    y = data[y_inds].values
    pixels = np.c_[x,y]
    left_image = frames_left[i]
    right_image = frames_right[i]
    disp_map = getDisparityMap(left_image, right_image)
    coordinate3D = pixelTo3DCameraCoord(left_image, disp_map, pixels)
    temp_x = [j['X'] for j in coordinate3D]
    temp_y = [j['Y'] for j in coordinate3D]
    temp_z = [j['Z'] for j in coordinate3D]
    df_new.iloc[i][x_inds] = temp_x
    df_new.iloc[i][y_inds] = temp_y
    df_new.iloc[i][z_inds] = temp_z
    
import os

outdir = os.path.dirname(os.path.dirname(h5files[0]))
filename = f'/{vid_id}_3D_{suffix}_leastereo.h5'

outname = outdir + filename
df_new.to_hdf(outname, key = 'leastereo')

Processing time: 0.0268
Processing time: 0.0272
Processing time: 0.0287
Processing time: 0.0275
Processing time: 0.0266
Processing time: 0.0266
Processing time: 0.0289
Processing time: 0.0279
Processing time: 0.0279
Processing time: 0.0272
Processing time: 0.0269
Processing time: 0.0268
Processing time: 0.0268
Processing time: 0.0281
Processing time: 0.0272
Processing time: 0.0284
Processing time: 0.0297
Processing time: 0.0301
Processing time: 0.0268
Processing time: 0.0280
Processing time: 0.0282
Processing time: 0.0282
Processing time: 0.0271
Processing time: 0.0281
Processing time: 0.0278
Processing time: 0.0268
Processing time: 0.0282
Processing time: 0.0279
Processing time: 0.0268
Processing time: 0.0272
Processing time: 0.0272
Processing time: 0.0269
Processing time: 0.0274
Processing time: 0.0269
Processing time: 0.0271
Processing time: 0.0285
Processing time: 0.0280
Processing time: 0.0267
Processing time: 0.0280
Processing time: 0.0287
Processing time: 0.0267
Processing time:

Processing time: 0.0279
Processing time: 0.0279
Processing time: 0.0296
Processing time: 0.0281
Processing time: 0.0288
Processing time: 0.0280
Processing time: 0.0279
Processing time: 0.0277
Processing time: 0.0273
Processing time: 0.0282
Processing time: 0.0285
Processing time: 0.0304
Processing time: 0.0279
Processing time: 0.0272
Processing time: 0.0280
Processing time: 0.0272
Processing time: 0.0299
Processing time: 0.0279
Processing time: 0.0271
Processing time: 0.0286
Processing time: 0.0278
Processing time: 0.0281
Processing time: 0.0273
Processing time: 0.0271
Processing time: 0.0276
Processing time: 0.0271
Processing time: 0.0273
Processing time: 0.0275
Processing time: 0.0269
Processing time: 0.0271
Processing time: 0.0271
Processing time: 0.0277
Processing time: 0.0268
Processing time: 0.0290
Processing time: 0.0273
Processing time: 0.0269
Processing time: 0.0272
Processing time: 0.0278
Processing time: 0.0286
Processing time: 0.0292
Processing time: 0.0280
Processing time:

In [77]:
df_new

scorer      DLC_dlcrnetms5_make_tea_pitcherJan29shuffle1_110000             \
individuals                                             pitcher              
bodyparts                                             bodypart1              
coords                                                        x          y   
0                                                   -32.787113  -32.787113   
1                                                   -38.638558  -38.638558   
2                                                   -65.137298  -65.137298   
3                                                   -57.336117  -57.336117   
4                                                   -66.719692  -66.719692   
..                                                         ...         ...   
533                                                 -26.642267  -22.519059   
534                                                 -26.534085  -22.427620   
535                                                 -26.473468  -22.502447   
536                                                 -26.465838  -22.495962   
537                                                 -26.437204  -22.471623   

scorer                                                                \
individuals                                                            
bodyparts                bodypart2                         bodypart3   
coords                z          x          y           z          x   
0             84.502870 -21.699552 -18.897687   59.614155 -23.517682   
1             99.583912 -21.676947 -18.878001   59.552053 -23.409472   
2            167.879633 -21.700747 -18.898727   59.617437 -23.447001   
3            147.773499 -21.668652 -18.870777   59.529265 -23.435885   
4            171.957969 -21.701872 -18.899708   59.620529 -23.459080   
..                  ...        ...        ...         ...        ...   
533           63.433969 -65.672160 -46.085726  164.591880 -57.433227   
534           63.176393 -55.754001 -39.125614  139.734337 -52.197031   
535           63.032066 -24.769777 -18.218576   62.392385 -45.129834   
536           63.013900 -24.774934 -18.222370   62.405376 -29.256210   
537           62.945723 -24.791527 -18.234574   62.447170 -28.462849   

scorer                                                                \
individuals                                                            
bodyparts                           bodypart4                          
coords               y           z          x          y           z   
0           -18.000337   68.966809 -23.038021 -12.572800   72.675146   
1           -17.917514   68.649479 -23.581070 -12.869164   74.388232   
2           -17.946239   68.759535 -51.251082 -27.969834  161.675338   
3           -17.937730   68.726935 -46.798400 -25.539821  147.629022   
4           -17.955484   68.794956 -37.978248 -20.726300  119.805200   
..                 ...         ...        ...        ...         ...   
533         -34.774639  157.351308 -23.831807 -11.143887   67.131852   
534         -31.604230  143.005565 -23.988760 -11.217280   67.573973   
535         -28.423675  116.014998 -31.772664 -15.307838   88.999058   
536         -18.426148   75.208765 -66.289094 -31.937603  185.683737   
537         -17.926473   73.169277 -65.420899 -31.519313  183.251817   

scorer                                         
individuals                                    
bodyparts    bodypart5                         
coords               x          y           z  
0           -43.457551 -15.952772  137.523896  
1           -48.805375 -17.915897  154.447388  
2           -23.403573  -8.591185   74.061939  
3           -39.862999 -14.633253  126.148730  
4           -43.708839 -16.045017  138.319110  
..                 ...        ...         ...  
533         -23.613501  -7.748180   73.792190  
534         -23.247346  -7.602906   73.104862  
535         -23.119732  -7.488467   72.703560  
536         -58.427737 -18.924707  183.7350

In [82]:
outname

'/home/luke/Desktop/project/make_tea/camera-main/videos/1642994619/left/1642994619_3D.h5'